# 기술 구현

In [ ]:
# 자율 주행에서 상해 정도와 관련된 변수들에 대한 분석을 진행한 논문
# 상해 정도가 클 수록 사고 발생 위험도가 크기 때문에, 이를 기반하여 가중치, 변수 설정

'''
MPC2 : 0.24, 타 차량 움직임
MPC1 : 0.22, 내 차량 움직임
Time : 0.14, 사고 발생 시간
TC2 : 0.11, 충돌 유형
Lighting : 0.10, 사고 발생시 조명
TC1 : 0.08, 충돌 유형
R_surface : 0.05, 도로 표면
Weather : 0.03, 사고 발생 날씨

MPC2-1 : 0.01
Other : 0.01
TC2-1 : 0.01
''' 

영향률이 0.01 이하였던 변수 제외 그리고 실제 사고가 아닌 위험도 측정을 위한 것이기 때문에 충돌 유형 제거하여
최종 변수 MPC2, MPC1, TIme, Lighting, R_surface, Weather 로 선정

-> 식으로 나타내면
risk = 0.24 * MPC2 + 0.22 * MPC1 + 0.14 * Time + 0.1 * Lighting + 0.05 * R_surface + 0.03 * Weather

In [ ]:
# 총 0.78 -> * 1.28 하면 됌

In [17]:
import cv2

cctv_file = 'vedio.mp4'
output_file = 'output.mp4'

cap = cv2.VideoCapture(cctv_file)

frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

MPC2 = 0.4
MPC1 = 0.4
Time = 0.3 # 고정
Lighting = 0.35
R_surface = 0.35
Weather = 0.3 # 고정


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

risk_list = []

while(cap.isOpened()):
    rst, frame = cap.read()
    
    if not rst:  # 프레임을 읽지 못한 경우
        print(all_cost)
        
        break

        
#############################################################################################################

    
    MPC2_dict = {35 : 0.37, 43 : 0.32, 67 : 0.3, 103 : 0.25, 140 : 0.2, 224 : 0.17, 270 : 0.25, 293 : 0.4, 
                 301 : 0.6, 314 : 0.4, 326 : 0.35, 348 : 0.4}  
    if frame_count in MPC2_dict.keys():
        MPC2 = MPC2_dict[frame_count]
      
        
    MPC1_dict = {40 : 0.53, 58 : 0.5, 83 : 0.45, 100 : 0.35, 130 : 0.3, 225 : 0.35, 273 : 0.3, 340 : 0.9}
    if frame_count in MPC1_dict.keys():
        MPC1 = MPC1_dict[frame_count]
        
        
    Lighting_dict = {190 : 0.3, 223 : 0.25, 350 : 0.4}
    if frame_count in Lighting_dict.keys():
        Lighting = Lighting_dict[frame_count]        

    R_surface_dict = {156 : 0.2, 187 : 0.25, 274 : 0.2, 299 : 0.25, 366 : 0.2}
    if frame_count in R_surface_dict.keys():
        R_surface = R_surface_dict[frame_count] 
    
    
    risk = 0.24 * MPC2 + 0.22 * MPC1 + 0.14 * Time + 0.1 * Lighting + 0.05 * R_surface + 0.03 * Weather
    risk = risk * 1.28 # 0~1 범위 맞추기 위해
    risk = round(risk, 3)
#############################################################################################################        
        
        
    risk_list.append(risk)
    
    risk_mean = round(sum(risk_list) / len(risk_list), 3)
    
        
        
    if risk > 0.5:
        cv2.rectangle(frame, (1525, 516), (1917, 650), (211, 211, 211), -1)
        cv2.rectangle(frame, (1525, 516), (1917, 650), (0, 0, 255), 3)
        cv2.putText(frame, 'Risk : ' + str(risk), (1560, 592), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 255), 4)
    else:
        cv2.rectangle(frame, (1525, 516), (1917, 650), (211, 211, 211), -1)
        cv2.rectangle(frame, (1525, 516), (1917, 650), (0, 128, 0), 3)
        cv2.putText(frame, 'Risk : ' + str(risk), (1560, 592), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 128, 0), 4)
        
    all_cost = round((0.5 + risk_mean) * 0.0339 * 300, 3)    
        
    if frame_count >= 0:
        cv2.rectangle(frame, (15, 920), (600, 1065), (211, 211, 211), -1)
        cv2.putText(frame, 'Risk mean : ' + str(risk_mean), (50, 970), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0,0,0), 4)
    
    if frame_count >= 390:
        cv2.putText(frame, 'cost : ' + str(all_cost), (50, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0,0,0), 4)
    
    cv2.namedWindow("Frame")
    cv2.imshow('Frame', frame)
    





    
    frame_count += 1
    
    out.write(frame)  # 동영상 프레임을 저장

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

8.502


In [5]:
frame_count

391

In [ ]:
## version 1 (ORG)

import cv2

cctv_file = 'vedio.mp4'
output_file = 'output.mp4'

cap = cv2.VideoCapture(cctv_file)

frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

MPC2 = 0.4
MPC1 = 0.4
Time = 0.3 # 고정
Lighting = 0.35
R_surface = 0.35
Weather = 0.3 # 고정


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))


while(cap.isOpened()):
    rst, frame = cap.read()
    
    if not rst:  # 프레임을 읽지 못한 경우
        break

        
#############################################################################################################

    
    MPC2_dict = {35 : 0.37, 43 : 0.32, 67 : 0.3, 103 : 0.25, 140 : 0.2, 224 : 0.17, 270 : 0.25, 293 : 0.4, 
                 301 : 0.6, 314 : 0.4, 326 : 0.35, 348 : 0.4}  
    if frame_count in MPC2_dict.keys():
        MPC2 = MPC2_dict[frame_count]
      
        
    MPC1_dict = {40 : 0.53, 58 : 0.5, 83 : 0.45, 100 : 0.35, 130 : 0.3, 225 : 0.35, 273 : 0.3, 340 : 0.9}
    if frame_count in MPC1_dict.keys():
        MPC1 = MPC1_dict[frame_count]
        
        
    Lighting_dict = {190 : 0.3, 223 : 0.25, 350 : 0.4}
    if frame_count in Lighting_dict.keys():
        Lighting = Lighting_dict[frame_count]        

    R_surface_dict = {156 : 0.2, 187 : 0.25, 274 : 0.2, 299 : 0.25, 366 : 0.2}
    if frame_count in R_surface_dict.keys():
        R_surface = R_surface_dict[frame_count] 
    
    
    risk = 0.24 * MPC2 + 0.22 * MPC1 + 0.14 * Time + 0.1 * Lighting + 0.05 * R_surface + 0.03 * Weather
    risk = risk * 1.28 # 0~1 범위 맞추기 위해
    risk = round(risk, 3)
#############################################################################################################        
        
        
        
        
        
    if risk > 0.5:
        cv2.rectangle(frame, (1530, 516), (1917, 650), (0, 0, 255), 2)
        cv2.putText(frame, 'Risk : ' + str(risk), (1570, 588), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
    else:
        cv2.rectangle(frame, (1530, 516), (1917, 650), (0, 128, 0), 2)
        cv2.putText(frame, 'Risk : ' + str(risk), (1570, 588), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 128, 0), 4)
    
    cv2.namedWindow("Frame")
    cv2.imshow('Frame', frame)
    
#     if frame_count % 10 == 0:
#         cv2.imwrite('frame_' + str(frame_count) + '.jpg', frame)




    
    frame_count += 1
    
    out.write(frame)  # 동영상 프레임을 저장

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()